In [1]:
# https://arxiv.org/abs/1610.02415

# https://pytorch-geometric.readthedocs.io/en/latest/notes/introduction.html

import torch
import torch.nn.functional as F
import torch.nn as nn
import torch.distributed as dist

import torch_geometric
import torch_geometric.nn as gnn

from torch_geometric.datasets import QM9
import GCL.augmentors
import GCL.augmentors as A
import edge_removing as A_alternate
from torch_geometric.nn import GCNConv

from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, mean_squared_error
from sklearn.linear_model import RidgeClassifierCV, LogisticRegression, LinearRegression
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
import lightgbm as lgb
from rdkit.Chem import PeriodicTable
from rdkit import Chem
from xenonpy.datatools import preset
from xenonpy.descriptor import Compositions
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn import preprocessing
from matplotlib.pylab import plt
from numpy import arange
import math
import timeit

In [2]:

 
#record start time
t_0 = timeit.default_timer()
# call function



In [3]:
parameters = {}
parameters['batch_size'] = 1000
periodic_table = Chem.GetPeriodicTable()

In [4]:
whole_dataset = QM9(root = 'data/')

#print(whole_dataset.get_summary())
#print(dir(whole_dataset))
#print(whole_dataset.len())
tr_ratio = 0.9
val_ratio = 0.09
test_ratio = 0.01


n = whole_dataset.len()
#print("n: ", n)
tr_n = math.floor(tr_ratio*n) # Number of QM9 to use as training data
val_n = math.floor(val_ratio*n)



all_inds = range(n)
#print("all_inds: ", all_inds)
tr_inds, val_inds = train_test_split(all_inds, train_size = tr_n)
val_test_inds = range(n - tr_n)
#print("val_test_inds: ", val_test_inds)
val_inds, test_inds = train_test_split(val_test_inds, train_size = val_n)


print("Size of training set: ", len(tr_inds))
print("Size of validation set: ", len(val_inds))
print("Size of test set: ", len(test_inds))
#print(type(tr_inds), type(tr_inds[0]))


train_sampler = torch.utils.data.SubsetRandomSampler(tr_inds)
val_sampler = torch.utils.data.SubsetRandomSampler(val_inds)
test_sampler = torch.utils.data.SubsetRandomSampler(test_inds)


# We need to make a train and validation set since QM9 does not provide them
train_set = torch.utils.data.Subset(whole_dataset, tr_inds)
val_set = torch.utils.data.Subset(whole_dataset, val_inds)
test_set = torch.utils.data.Subset(whole_dataset, test_inds)

train_loader = torch_geometric.loader.DataLoader(train_set, batch_size = parameters['batch_size'],
                                                shuffle = True, num_workers = 2,)
                                                #sampler = train_sampler)
big_train_loader = torch_geometric.loader.DataLoader(train_set, batch_size = int(1e9),
                                                shuffle = True, num_workers = 2,)

val_loader = torch_geometric.loader.DataLoader(val_set, batch_size=200,
                                            shuffle=True, num_workers=2,)
                                              #sampler = val_sampler)
test_loader = torch_geometric.loader.DataLoader(test_set, batch_size=100,
                                            shuffle=True, num_workers=2,)
                                              #sampler = val_sampler)

Size of training set:  117747
Size of validation set:  11774
Size of test set:  1310


In [5]:
test_graph_chem_formulae_dictionaries = pd.DataFrame()
tr_mol_list = []

In [6]:
qm9_index = {0: 'Dipole moment',
1: 'Isotropic polarizability',
2: 'Highest occupied molecular orbital energy',
3: 'Lowest unoccupied molecular orbital energy',
4: 'Gap between previous 2',
5: 'Electronic spatial extent',
6: 'Zero point vibrational energy',
7: 'Internal energy at 0K',
8: 'Internal energy at 298.15K',
9: 'Enthalpy at 298.15K',
10: 'Free energy at 298.15K',
11: 'Heat capacity at 298.15K',
12: 'Atomization energy at 0K',
13: 'Atomization energy at 298.15K',
14: 'Atomization enthalpy at 298.15K',
15: 'Atomization free energy at 298.15K',
16: 'Rotational constant A',
17: 'Rotational constant B',
18: 'Rotational constant C'}

qm9_index_list = ['Dipole moment', 
                  'Isotropic polarizability',
                  'Highest occupied molecular orbital energy',
                  'Lowest unoccupied molecular orbital energy',
                  'Gap between previous 2',
                  'Electronic spatial extent',
                  'Zero point vibrational energy',
                  'Internal energy at 0K',
                  'Internal energy at 298.15K',
                  'Enthalpy at 298.15K',
                  'Free energy at 298.15K',
                  'Heat capacity at 298.15K',
                  'Atomization energy at 0K',
                  'Atomization energy at 298.15K',
                  'Atomization enthalpy at 298.15K',
                  'Atomization free energy at 298.15K',
                  'Rotational constant A',
                  'Rotational constant B',
                  'Rotational constant C']

In [7]:
x_index = {0: 'H atom?',
1: 'C atom?',
2: 'N atom?',
3: 'O atom?',
4: 'F atom?',
5: 'atomic_number',
6: 'aromatic',
7: 'sp1',
8: 'sp2',
9: 'sp3',
10: 'num_hs'}
x_index_list = ['H atom?', 
                'C atom?', 
                'N atom?', 
                'O atom?', 
                'F atom?', 
                'atomic_number', 'aromatic', 
                'sp1',
                'sp2',
                'sp3',
                'num_hs']

In [8]:
def off_diagonal(x):
    n, m = x.shape
    assert n == m
    return x.flatten()[:-1].view(n - 1, n + 1)[:, 1:].flatten()

def VicRegLoss(x, y):
    # https://github.com/facebookresearch/vicreg/blob/4e12602fd495af83efd1631fbe82523e6db092e0/main_vicreg.py#L184
    # x, y are output of projector(backbone(x and y))
    repr_loss = F.mse_loss(x, y)

    x = x - x.mean(dim=0)
    y = y - y.mean(dim=0)

    std_x = torch.sqrt(x.var(dim=0) + 0.0001)
    std_y = torch.sqrt(y.var(dim=0) + 0.0001)
    std_loss = torch.mean(F.relu(1 - std_x)) / 2 + torch.mean(F.relu(1 - std_y)) / 2

    cov_x = (x.T @ x) / (parameters['batch_size'] - 1)
    cov_y = (y.T @ y) / (parameters['batch_size'] - 1)
    cov_loss = off_diagonal(cov_x).pow_(2).sum().div(
        x.shape[1]
    ) + off_diagonal(cov_y).pow_(2).sum().div(x.shape[1])
    
    # self.num_features -> rep_dim?
    loss = (
        sim_coeff * repr_loss
        + std_coeff * std_loss
        + cov_coeff * cov_loss
    )
    return loss


def atoms_dictionary(atomic_num):
    #print("atomic_num: ", atomic_num)
    atomic_symbol = periodic_table.GetElementSymbol(atomic_num)
    return atomic_symbol



In [9]:
def XenonPy_transform(df, df_dict_column):
    cal = Compositions()
    comps = df[df_dict_column]
    descriptors = cal.transform(comps)
    column_names = list(descriptors.columns)
    scaler = preprocessing.StandardScaler().fit(descriptors)
    descriptors = scaler.transform(descriptors)
    descriptors = pd.DataFrame(descriptors, columns = column_names)
    return(descriptors)

In [10]:
def get_mol_dict(batch):
    
    graph_chem_formulae_dictionaries = pd.DataFrame()
    if not graph_chem_formulae_dictionaries.empty:
        graph_chem_formulae_dictionaries.drop(columns = 'formula')

    node_to_graph_indicator = pd.DataFrame(batch.batch).astype("int")
    node = pd.DataFrame(batch.x).astype("int")
    mol_list = []
    j = 0
    mol_dict = {}
    for i in range(len(batch.z)):
            #get a dictionary for each graph that contains chemical formula
                #format for use for XenonPy
        if j == int(node_to_graph_indicator.iloc[i]):
                #add this ith atom to to the dictionary for the jth graph
                #atoms_dictionary(atomic_num)
                #call function to add element to molecular dictionary
            element = atoms_dictionary(int(node[5].iloc[i]))
            if element in mol_dict:
                mol_dict[element] = mol_dict[element] + 1
            else:
                mol_dict[element] = 1
        else: #need to move to next graph
                #Insert these dictionaries to each row in the df
            mol_list.append(mol_dict)
            mol_dict = {}
            element = atoms_dictionary(int(node[5].iloc[i]))
            j += 1

    mol_list.append(mol_dict) #need to append the last dict
    graph_chem_formulae_dictionaries.insert(0, 'formula', mol_list)
    for i in range(len(batch.y) - 1):
        if mol_list[i]:
            pass
        else:
            print("Empty!!", " location: ", i)

    return graph_chem_formulae_dictionaries

In [11]:
class GCN(torch.nn.Module):
    def __init__(self):
        super().__init__()
        
        self.rep_dim = 128
        self.emb_dim = 256
        
        # Data under graph
        self.conv1 = GCNConv(whole_dataset.num_node_features, self.rep_dim // 2)
        self.bn1 = nn.BatchNorm1d(self.rep_dim // 2)
        self.a1 = nn.LeakyReLU(0.02)
        
        self.conv2 = GCNConv(self.rep_dim // 2, self.rep_dim) # To Rep Space
        self.bn2 = nn.BatchNorm1d(self.rep_dim)
        
        # Projection to representation
        self.mpool1 = gnn.global_mean_pool
        #self.fc1 = nn.Linear(self.rep_dim, self.rep_dim)
        
        # Graph 2
        self.conv3 = GCNConv(self.rep_dim, self.rep_dim * 2) # To Emb Space
        self.bn3 = nn.BatchNorm1d(self.rep_dim * 2)
        
        # Projection to embedding
        self.mpool2 = gnn.global_mean_pool
        self.fc2 = nn.Linear(self.emb_dim, self.emb_dim) # Linear to rep?
            #might want to get rid of this
        
    def forward(self, data, binds):
        x = data[0].float().to(device)
        edge_index = data[1].to(device)
        
        # Input graph to GConv
        x = self.conv1(x, edge_index)
        x = self.a1(self.bn1(x))
        x = F.dropout(x, training=self.training)
        
        x = self.bn2(self.conv2(x, edge_index))
        
        # GConv outputs projected to representation space
        #print('before pool: ', x.shape)
        x_rep = self.mpool1(x, binds)
        #print('pooled: ', x_rep.shape)
        
        #x_rep = self.fc1(x_rep)
        #print('projected: ', x_rep.shape, 'gconv', x.shape)
        
        x_emb = self.bn3(self.conv3(x, edge_index))
        #print('x emb after conv3', x_emb.shape)
        x_emb = self.mpool2(x_emb, binds)
        #print('after pool', x_emb.shape)
        x_emb = self.fc2(x_emb)
        #print('after fc2', x_emb.shape)
        
        return x_rep, x_emb

In [12]:
device = 'cuda'

model = GCN().to(device)

sim_coeff = 25
std_coeff = 25
cov_coeff = 1

aug = A.RandomChoice([#A.RWSampling(num_seeds=1000, walk_length=10),
                      A.NodeDropping(pn=0.1),
                      A.FeatureMasking(pf=0.1),
                      A_alternate.EdgeRemoving(pe=0.1)], #edge_adj was deprecated, so need to use edge_ something instead
                      num_choices=1)
#should do many other types of augmentations
    #train models on all but one augmentations and see which work best
        #ablation study!
val_aug = A.RandomChoice([], num_choices = 0)
optimizer = torch.optim.Adam(model.parameters(), lr=0.002, weight_decay=5e-4)

In [13]:
#for atom in mol.GetAtoms():
                #type_idx.append(types[atom.GetSymbol()])
#pseudocode to get molecule's chemical formula/SMILES/etc.

In [14]:
def plot_loss_curves(loss_per_epoch, val_loss):
    train_values = loss_per_epoch
    val_values = val_loss
 
    # Generate a sequence of integers to represent the epoch numbers
    epochs = range(0, len(loss_per_epoch))
 
    # Plot and label the training and validation loss values
    plt.plot(epochs, train_values, label='Training Loss')
    plt.plot(epochs, val_values, label='Validation Loss')
 
    # Add in a title and axes labels
    plt.title('Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
 
    # Set the tick locations

    plt.xticks(arange(0, len(loss_per_epoch), max(math.floor(len(loss_per_epoch)/10), 1)))
 
    # Display the plot
    plt.legend(loc='best')
    plt.show()

In [ ]:
tr_graph_chem_formulae_dictionaries = pd.DataFrame()
n_epochs = 100
epoch_loss = []
val_epoch_loss = []
print("Start training!")
for epoch in range(0,n_epochs+1):
    #print("epoch: ", epoch)
    epoch_losses = []
    for batch in train_loader:
        optimizer.zero_grad()

        batch_inds = batch.batch.to(device)
      
            
        if epoch == 0: #gets dictionary of constituent atoms for each graph for XenonPy
            tr_graph_chem_formulae_dictionaries = pd.concat([tr_graph_chem_formulae_dictionaries, get_mol_dict(batch)])     
        
        # batch of graphs has edge attribs, node attribs - (n_nodes, n_features+1) -> concat (n_nodes, attrib1)

        batch.x = batch.x.float()#.to(device)
        #batch.edge_index = batch.edge_index.to(device)
        #print("new_batch: ", batch.z)
        # Barlow - get 2 random views of batch
        b1 = aug(batch.x, batch.edge_index, batch.edge_attr)
        b2 = aug(batch.x, batch.edge_index, batch.edge_attr)

        # Embed each batch (ignoring representations)
        r1, e1 = model(b1, batch_inds)
        r2, e2 = model(b2, batch_inds)

        loss = VicRegLoss(e1, e2)
        loss.backward()
        optimizer.step()

        epoch_losses.append(loss.data.item())
        
    #epoch_loss.append(sum(epoch_losses) / len(epoch_losses))
    #print('epoch', epoch,'train loss:', sum(epoch_losses) / len(epoch_losses))
    epoch_loss.append(sum(epoch_losses) / len(tr_inds)) 
    print('epoch', epoch,'train loss:', sum(epoch_losses) / len(tr_inds))
    
    
    val_loss = 0
    model.eval()
    
    # VicReg Validation Loss
    val_epoch_losses = []
    for batch in val_loader:
        with torch.no_grad():
            # VicReg validation loss
            b1 = aug(batch.x, batch.edge_index, batch.edge_attr)
            b2 = aug(batch.x, batch.edge_index, batch.edge_attr)
            r1, e1 = model(b1, batch.batch.to(device))
            r2, e2 = model(b2, batch.batch.to(device))
                
            val_loss = VicRegLoss(e1, e2)
            val_epoch_losses.append(val_loss.data.item())
            

    #val_epoch_loss.append(sum(val_epoch_losses) / len(val_epoch_losses))    
    #print('epoch', epoch,'val loss:', sum(val_epoch_losses) / len(val_epoch_losses))
    val_epoch_loss.append(sum(val_epoch_losses) / len(val_inds))    
    print('epoch', epoch,'val loss:', sum(val_epoch_losses) / len(val_inds))
    
    
    
    if epoch == n_epochs:
        print("Done augmenting!")
        
       
        
        

Start training!
epoch 0 train loss: 0.01982753290411613
epoch 0 val loss: 0.11311989533631896
epoch 1 train loss: 0.02757869398979533
epoch 1 val loss: 0.09801480060160737
epoch 2 train loss: 0.01993731382792247
epoch 2 val loss: 0.09222302194643611
epoch 3 train loss: 0.02026521859191725
epoch 3 val loss: 0.09294607288184992
epoch 4 train loss: 0.019436121434427637
epoch 4 val loss: 0.08840640437244579
epoch 5 train loss: 0.01983770996579452
epoch 5 val loss: 0.0868481967654519
epoch 6 train loss: 0.019185237257145733
epoch 6 val loss: 0.085824466184989
epoch 7 train loss: 0.01914094796325258
epoch 7 val loss: 0.08783745818334948
epoch 8 train loss: 0.01903340152087768
epoch 8 val loss: 0.08481779555341314
epoch 9 train loss: 0.019108123301111175
epoch 9 val loss: 0.08711187345823246
epoch 10 train loss: 0.01901530303445791
epoch 10 val loss: 0.08774413069460496
epoch 11 train loss: 0.01972957013254667
epoch 11 val loss: 0.08858796649904997


In [ ]:
print(epoch_loss)

In [ ]:
plot_loss_curves(epoch_loss, val_epoch_loss)

In [ ]:
#tr_df_XenonPy = XenonPy_transform(tr_graph_chem_formulae_dictionaries, 'formula')

In [ ]:
#print(tr_df_XenonPy)

In [ ]:
#Get embedded training set

x_tr = pd.DataFrame()
x_tr_no_aug = pd.DataFrame()
y_tr = pd.DataFrame()
y_tr_no_aug = pd.DataFrame()
for batch in train_loader: # take entire train set
    print("batch.x.shape: ", batch.x.shape)
    x_tr_tabular_no_aug = pd.DataFrame(batch.x).astype("float")
    x_tr_no_aug = pd.concat([x_tr_no_aug, x_tr_tabular_no_aug])
    y_tr_tabular_no_aug = pd.DataFrame(batch.y).astype("float")
    y_tr_no_aug = pd.concat([y_tr_no_aug, y_tr_tabular_no_aug])
    with torch.no_grad():
        # Embed training set under model
        rep_tr, _ = model(val_aug(batch.x, batch.edge_index, batch.edge_attr), batch.batch.to(device))
        if torch.cuda.is_available():
            rep_tr = rep_tr.to("cpu")
        rep_tr_tabular = pd.DataFrame(rep_tr.numpy())
        x_tr = pd.concat([x_tr, rep_tr_tabular])
        y_tr_tabular = pd.DataFrame(batch.y).astype("float")
        y_tr = pd.concat([y_tr, y_tr_tabular])
        
        
x_val = pd.DataFrame()
x_val_no_aug = pd.DataFrame()
y_val = pd.DataFrame()
y_val_no_aug = pd.DataFrame()
val_graph_chem_formulae_dictionaries = pd.DataFrame()
for val_batch in val_loader:
    print("val_batch.x.shape: ", val_batch.x.shape)
    x_val_tabular_no_aug = pd.DataFrame(val_batch.x).astype("float")
    print("x_val_tabular_no_aug: ", x_val_tabular_no_aug)
    x_val_no_aug = pd.concat([x_val_no_aug, x_val_tabular_no_aug])
    print("x_val_no_aug: ", x_val_no_aug)
    y_val_tabular_no_aug = pd.DataFrame(val_batch.y).astype("float")
    y_val_no_aug = pd.concat([y_val_no_aug, y_val_tabular_no_aug])
    val_graph_chem_formulae_dictionaries = pd.concat([val_graph_chem_formulae_dictionaries, get_mol_dict(val_batch)])
    with torch.no_grad():
        # Embed validation set under model
        rep_val, _ = model(val_aug(val_batch.x, val_batch.edge_index, val_batch.edge_attr), val_batch.batch.to(device))
        if torch.cuda.is_available():
            rep_val = rep_val.to("cpu")
        rep_val_tabular = pd.DataFrame(rep_val.numpy())
        x_val = pd.concat([x_val, rep_val_tabular])
        print("x_val: ", x_val)
        y_val_tabular = pd.DataFrame(val_batch.y).astype("float")
        y_val = pd.concat([y_val, y_val_tabular])
        print("y_val: ", y_val)

x_test = pd.DataFrame()
x_test_no_aug = pd.DataFrame()
y_test = pd.DataFrame()
y_test_no_aug = pd.DataFrame()                
for test_batch in test_loader:
    x_test_tabular_no_aug = pd.DataFrame(test_batch.x).astype("float")
    x_test_no_aug = pd.concat([x_test_no_aug, x_test_tabular_no_aug])
    y_test_tabular_no_aug = pd.DataFrame(test_batch.y).astype("float")
    y_test_no_aug = pd.concat([y_test_no_aug, y_test_tabular_no_aug])
    test_graph_chem_formulae_dictionaries = pd.concat([test_graph_chem_formulae_dictionaries, get_mol_dict(test_batch)])
    with torch.no_grad():
        # Embed validation set under model
        rep_test, _ = model(test_aug(test_batch.x, test_batch.edge_index, test_batch.edge_attr), test_batch.batch.to(device))
        if torch.cuda.is_available():
            rep_test = rep_test.to("cpu")
        rep_test_tabular = pd.DataFrame(rep_test.numpy())
        x_test = pd.concat([x_test, rep_test_tabular])
        y_test_tabular = pd.DataFrame(test_batch.y).astype("float")
        y_test = pd.concat([y_test, y_test_tabular])



In [ ]:

print("x_val_no_aug: ", x_val_no_aug)
print("y_val_no_aug: ", y_val_no_aug.shape)
print("x_val: ", x_val)
print("y_val: ", y_val)

In [ ]:
#val_df_XenonPy = XenonPy_transform(val_graph_chem_formulae_dictionaries, 'formula')

In [ ]:
print(x_val_no_aug)

In [ ]:
#val_df_XenonPy.rename(lambda x: str(x), axis='columns')
#val_df_XenonPy.columns = val_df_XenonPy.columns.astype(str)

In [ ]:
#for i in val_df_XenonPy:
    #print(i)

In [ ]:
#a function to run linear models
def linear_models(x_train, x_test, y_train, y_test, list_target_features):
    # For each task in QM9
    means_vector = y_train.mean(axis = 0)
    rep_means_vectors = means_vector.repeat(x_train.shape[0]) #create a vector where each entry is the mean
    for target_feature in range(y_test.shape[1]):

        # Fit a model on model representation of train set:
        #lm = LinearRegression().fit(x_train.values, y_train[target_feature].values)
        
        #Fit Random Forest models here:
        #rf = RandomForestRegressor(n_estimators=10, max_depth=10 )
        #rf.fit(x_train, y_train[target_feature])
        #rf_yhat = rf.predict(x_test)
        
        #Fit LightGBM models here (LightGBM is supposedly better than XGBoost):
        lgb_train = lgb.Dataset(x_train.values, y_train[target_feature].values, params={'verbose': -1})
        lgb_eval = lgb.Dataset(x_test.values, y_test[target_feature].values, reference=lgb_train, params={'verbose': -1})
        params = {
            'boosting_type': 'gbdt',
            'objective': 'regression',
            'metric': {'l2', 'l1'},
            'num_leaves': 31,
            'learning_rate': 0.05,
            'force_col_wise': 'true',
            'feature_fraction': 0.9,
            'bagging_fraction': 0.8,
            'bagging_freq': 5,
            'verbose': -1
        }
        
        gbm = lgb.train(params,
                        lgb_train,
                        num_boost_round=20,
                        valid_sets=lgb_eval,
                        callbacks=[lgb.early_stopping(stopping_rounds=5)])
        lgb_yhat = gbm.predict(x_test.values, num_iteration=gbm.best_iteration)
        
        
        # Test the model on model repersentation of val set
        #yhat = lm.predict(x_test.values)
        #score = mean_squared_error(y_test[target_feature].values, yhat)
        #rf_score = mean_squared_error(y_test[target_feature], rf_yhat)
        lgb_score = mean_squared_error(y_test[target_feature].values, lgb_yhat)
        rep_means_vectors = means_vector[target_feature].repeat(x_test.shape[0])
        baseline = mean_squared_error(y_test[target_feature].values, rep_means_vectors)
        #baseline is a model that always outputs the mean of the training sample
        print("Baseline MSE for ", list_target_features[target_feature], ": ", baseline)
        #print("Linear Regression Model MSE for ", list_target_features[target_feature], ": ", score)
        #print("RF Model Mean-Squared-Error for ", list_target_features[target_feature], ": ", rf_score)
        print("LightGBM Model MSE for ", list_target_features[target_feature], ": ", lgb_score)
        
        
        
        

In [ ]:
#need to make sure that I am getting the correct graph for y's

In [ ]:
#concatenate XenonPy transformations with x_tr and x_val
#x_tr = pd.concat([x_tr, tr_df_XenonPy])
#x_val = pd.concat([x_val, val_df_XenonPy])

In [ ]:
x_tr = pd.concat([x_tr, x_val]) #tr and val combined for training set
y_tr = pd.concat([x_tr, x_val]) #tr and val combined for training set

In [ ]:
linear_models(x_tr, x_test, y_tr, y_test, qm9_index)

In [ ]:
#linear_models(x_tr_no_aug, x_val_no_aug, y_tr_no_aug, y_val_no_aug, qm9_index)

In [ ]:
print("Hello, world!")

In [ ]:
# record end time
t_1 = timeit.default_timer()
 
# calculate elapsed time and print


In [ ]:
elapsed_time = round((t_1 - t_0) , 1)
print(f"Elapsed time: {elapsed_time} seconds")
elapsed_time_minutes = round((elapsed_time/60), 2)
print(f"Elapsed time: {elapsed_time_minutes} minutes")
elapsed_time_hours = round((elapsed_time/3600), 2)
print(f"Elapsed time: {elapsed_time_hours} hours")